<a href="https://colab.research.google.com/github/amir-10/Projet_Big_Data/blob/main/Projet_Big_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>